In [ ]:
# Install OpenAI and Flumes SDK up-front so downstream imports see them
%pip install -q openai
%pip install -q flumes-ai
print("Deps installed: openai + flumes-ai")


In [ ]:
# Ensure the OpenAI wrapper sees the installed package in this runtime
import importlib
import flumes.utils.openai_wrapper as _ow
importlib.reload(_ow)
print("OpenAI wrapper reloaded")


# Flumes: First Steps 🚀

Give your agents durable, structured memory in minutes. This notebook mirrors a realistic onboarding flow:

- Initialize the Python SDK
- Add a rich, real-world message and watch Flumes extract multiple memories (facts, events, preferences)
- Retrieve relevant context for a follow‑up question

### What makes Flumes different (at a glance)
- Single call to assemble context optionally extracts + stores the turn (no extra summarization step needed)
- Hybrid retrieval with tunable weights and sensible presets (balanced, factual, recent, graphy)
- Built‑in budgeting with small, predictable caps (light / standard / heavy) and returned pack telemetry
- Entity‑first design: you work with `entity_id` and optional `namespace` everywhere

You only need your Flumes API key. The base URL is built‑in: `https://api.flumes.ai`


In [ ]:
# Paste your API key here or set FLUMES_API_KEY in your environment
import os
API_KEY = os.getenv("FLUMES_API_KEY", "your-api-key")
if API_KEY == "your-api-key":
    print("⚠️ Set FLUMES_API_KEY or edit API_KEY above before running the rest.")


In [ ]:
# Install the SDK (one-time in Colab/local)
# If already installed, you can skip this cell
%pip install -q flumes-ai
print("Installed flumes-ai")


Installed flumes-ai


In [23]:
# Initialize the client (entity-first)
from flumes import MemoryClient

client = MemoryClient(api_key=API_KEY, agent_id="onboarding-bot")
entity_id = "user_demo_001"
namespace = "prod"

print("Client ready → entity:", entity_id, "namespace:", namespace)


Client ready → entity: user_demo_001 namespace: prod


In [24]:
# Step 1: Add a rich message (Flumes will extract structured facts/events)
# This text is intentionally dense so the graph has plenty to show.

u = client.for_entity(entity_id, namespace=namespace)

rich_message = (
    "I'm Alice Parker, living in Paris since 2023. I usually travel with my brother Ben. "
    "For work, I'm a product manager at Nova Labs. I prefer museums over nightlife and "
    "love quiet wine bars in the Latin Quarter. My favorite artists are Monet and Van Gogh. "
    "I'm training for a half-marathon; my pace is around 5:30/km. Budget for trips is about €1200. "
    "I have a gluten allergy and avoid crowded places on weekends. The last hotel I loved was Hotel Lumière "
    "near Jardin du Luxembourg; I rated it 5/5 in May 2025. I'm planning a 3-day getaway to Lyon next month, "
    "and I'd like recommendations that include scenic runs by the river, museums, and quiet dinner spots."
)

res = u.add(rich_message, budget="standard")
print("Stored. request_id=", res.get("request_id"))
print("Pack:", res.get("pack"))
print("Summary:\n", (res.get("context") or {}).get("summary", ""))


{"ts": "2025-08-19T16:28:04.166Z", "event": "request", "data": {"method": "POST", "url": "/v0/context/assemble", "headers": {"X-Flumes-Agent": "onboarding-bot", "Idempotency-Key": "b4eea01c74e9c7d4b857250209690d9d"}}}
{"ts": "2025-08-19T16:28:09.575Z", "event": "response", "data": {"status": 200, "url": "/v0/context/assemble", "request_id": "req_16749cc44ee947d49ce8697906f146cb"}}
Stored. request_id= req_16749cc44ee947d49ce8697906f146cb
Pack: {'target_tokens': 1200, 'hard_cap_tokens': 1500, 'used_tokens': 212, 'dropped': [{'action': 'dedupe_exact_text', 'memory_id': 'mem_b9e73e0f4c9a482f9612ef0cf862733e'}, {'action': 'dedupe_exact_text', 'memory_id': 'mem_1da0b5b4b71d48209dda0455f98a00e2'}, {'action': 'dedupe_exact_text', 'memory_id': 'mem_59dd668cfde2472987f7a770f9ce82bd'}, {'action': 'dedupe_exact_text', 'memory_id': 'mem_a1ef624c830647cca46514bd480e73a0'}, {'action': 'dedupe_exact_text', 'memory_id': 'mem_7917a06081b04c1593c9130ad27c2822'}, {'action': 'dedupe_exact_text', 'memory_id

In [25]:
# Step 2: Verify memories exist (facts/events)
page = client.get_all(entity_id=entity_id, namespace=namespace, limit=10)
print("Fetched", len(page.get("items", [])), "items")
for m in page.get("items", [])[:6]:
    print("-", m.get("type"), "::", m.get("text"))


{"ts": "2025-08-19T16:28:09.587Z", "event": "request", "data": {"method": "GET", "url": "/v0/memories", "params": {"entity_id": "user_demo_001", "namespace": "prod", "limit": 10}}}
{"ts": "2025-08-19T16:28:09.785Z", "event": "response", "data": {"status": 200, "url": "/v0/memories", "request_id": null}}
Fetched 10 items
- event :: I'm Alice Parker, living in Paris since 2023. I usually travel with my brother Ben. For work, I'm a product manager at Nova Labs. I prefer museums over nightlife and love quiet wine bars in the Latin Quarter. My favorite artists are Monet and Van Gogh. I'm training for a half-marathon; my pace is around 5:30/km. Budget for trips is about €1200. I have a gluten allergy and avoid crowded places on weekends. The last hotel I loved was Hotel Lumière near Jardin du Luxembourg; I rated it 5/5 in May 2025. I'm planning a 3-day getaway to Lyon next month, and I'd like recommendations that include scenic runs by the river, museums, and quiet dinner spots.
- fact :: tr

In [26]:
# Step 3: Retrieve context for a new message
query = "Looking for a quiet dinner near a scenic running route in Lyon; any museum picks?"
res = client.search(query, entity_id=entity_id, namespace=namespace, top_k=16, include_scores=True)
print("Matches:", len(res.get("matches", [])))
for m in res.get("matches", [])[:6]:
    print(f"- {m.get('text')} (score={round(m.get('score', 0),3)})")


{"ts": "2025-08-19T16:28:09.793Z", "event": "request", "data": {"method": "POST", "url": "/v0/recall"}}
{"ts": "2025-08-19T16:28:10.984Z", "event": "response", "data": {"status": 200, "url": "/v0/recall", "request_id": "req_0e9ca5d68c4a4d16ac88f5c2b5c4aa9b"}}
Matches: 16
- needs_recommendation = scenic runs by the river, museums, and quiet dinner spots (score=0.6)
- preference = quiet wine bars in the Latin Quarter (score=0.489)
- planning_trip = Lyon (score=0.469)
- trip_planning = Lyon (score=0.462)
- preferred_activity = quiet wine bars (score=0.381)
- preference = museums (score=0.375)


### Next steps
- Try adding another turn (preferences, places, ratings)
- Use the Agent helper for end-to-end chat with citations
- Explore the graph view in the dashboard to see entities and relationships emerge


## Advanced: Retrieval weights, budgeting, and presets

Flumes retrieval combines semantic, keyword (BM25), and graph signals. You control it with a simple `retrieval` object:

- `preset`: one of `balanced | factual | recent | graphy`
- Or `weights`: `{ semantic, bm25, graph_prior, recency_decay, confidence }` (server renormalizes; flags indicate if adjusted)
- `top_k`: number of candidates to return (default 16)

Budgeting keeps cost predictable:
- `budget = "light" | "standard" | "heavy"` → max_context_tokens of ~400 / 1200 / 2400 (hard caps applied)
- Each `add()` returns `pack = { target_tokens, hard_cap_tokens, used_tokens, dropped[] }`

You can pass these knobs per call or set them as defaults on the client.


In [27]:
# Try custom retrieval weights and a different budget
query = "Quiet dinner near scenic runs and impressionist museums in Lyon"
res = client.search(
    query,
    entity_id=entity_id,
    namespace=namespace,
    top_k=16,
    weights={"semantic": 0.6, "bm25": 0.3, "graph_prior": 0.1},
    include_scores=True,
)
print("Top 5 with custom weights:")
for m in res.get("matches", [])[:5]:
    print(f"- {m.get('text')} (score={round(m.get('score', 0),3)})")


{"ts": "2025-08-19T16:28:10.990Z", "event": "request", "data": {"method": "POST", "url": "/v0/recall"}}
{"ts": "2025-08-19T16:28:12.363Z", "event": "response", "data": {"status": 200, "url": "/v0/recall", "request_id": "req_82e8bcaa79df499b87d6794b94accc38"}}
Top 5 with custom weights:
- needs_recommendation = scenic runs by the river, museums, and quiet dinner spots (score=0.6)
- planning_trip = Lyon (score=0.526)
- trip_planning = Lyon (score=0.525)
- preference = quiet wine bars in the Latin Quarter (score=0.486)
- location = Paris (score=0.408)


## Optional: Agent SDK (end-to-end chat)
If you have an OpenAI key configured, you can run a short chat with grounded context. This uses the same `add()` assemble call under the hood.

Before running the next cells, ensure `OPENAI_API_KEY` is set in your environment.


In [36]:
# Install OpenAI (idempotent) before importing Agent
%pip install -q openai

# Reload the wrapper so it detects the installed package in this runtime
import importlib
import flumes.utils.openai_wrapper as _ow
importlib.reload(_ow)

from flumes import Agent


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# (Duplicate cell removed; openai already installed and wrapper reloaded above)


Note: you may need to restart the kernel to use updated packages.


In [35]:

agent = Agent(agent_id="onboarding-bot", entity_id=entity_id, memory_client=client)
print(agent.chat("Plan a quiet, museum-focused day in Lyon with a scenic morning run."))


RuntimeError: openai package not installed; run `pip install openai`. 

## Wrap‑up and next steps
You're done! You installed the SDK, initialized a client, added a dense message, verified stored memories, and retrieved context. 

Where to go next:
- View your memory graph in the dashboard (entities, facts, relationships)
- Read the API reference and concepts (retrieval presets, budgeting, policies)
- Explore SDK examples (Python, Node)

Replace these with your links:
- Dashboard: <YOUR_DASHBOARD_URL>
- Docs: <YOUR_DOCS_URL>
- SDK repos: <YOUR_SDK_URLS>
